# Searching for Be stars in binaries with compact objects
## Catalog cross-matching
### Caden Gobat, The George Washington University

Some conventions/names:
- The first letter of `astropy.table` object variable names are capitalized
- First letters of `pandas.DataFrame` names are lowercase
- CXO, CSC = Chandra X-ray Observatory, Chandra Source Catalog
- XMM, SSC = XMM-Newton, Serendipitous Source Catalog
- XRT, XPS = (Swift) X-ray Telescope, XRT Point Source (catalog)
- BeSS = Be Star Spectra database

In [1]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt, astropy.units as u, pickle, seaborn as sns
from mpl_toolkits import mplot3d
from scipy import stats
from PyAstronomy import pyasl
from astropy.io import fits, ascii as ascii_io
from astropy.coordinates import SkyCoord, Angle
from astroquery.vizier import Vizier
from astroquery.simbad import Simbad

from tools import hms_to_dec, dms_to_dec

### Initialize ViZier query instance

In [2]:
v = Vizier(columns=['all', '_RAJ2000', '_DEJ2000', "_r"])
v.ROW_LIMIT = -1 # no row limit

### Load the [Be star database](http://basebe.obspm.fr/basebe/)

In [3]:
BeSS = ascii_io.read("./catalogs/BeSS_catalog.csv", format="csv", header_start=0, data_start=1)
BeSS["_RAJ2000"] = [hms_to_dec(hms) for hms in BeSS["RA"]]*u.degree
BeSS["_DEJ2000"] = [dms_to_dec(dms) for dms in BeSS["DEC"]]*u.degree
BeSS.info

<Table length=2264>
         name          dtype  unit    class     n_bad
--------------------- ------- ---- ------------ -----
                    #   int32            Column     0
              Be star   str26            Column     0
             Category   str19            Column     0
                   RA   str11            Column     0
                  DEC   str12            Column     0
                    V float64      MaskedColumn   310
                 Type   str10      MaskedColumn   110
                vsini   int32      MaskedColumn  1648
Nb of spectra in BeSS   int32            Column     0
             _RAJ2000 float64  deg       Column     0
             _DEJ2000 float64  deg       Column     0

### Crossmatch with Gaia data

Create `gaia_df`, a `pandas.DataFrame` containing the single closest match (within $1^{\prime\prime}$) in the Gaia catalog for every entry in the BeSS database.

In [4]:
Gaia_BeSS_1as = v.query_region(BeSS, catalog="I/350/gaiaedr3", radius="1s")
Gaia_matched = Gaia_BeSS_1as[0]["_q","_r","EDR3Name","Plx","pmRA","pmDE","epsi","sepsi","Gmag","BPmag","RPmag","RVDR2","GLON","GLAT","Tefftemp"]

gaia_df = Gaia_matched.to_pandas()
for i in BeSS["#"]:
    instances = len(gaia_df[gaia_df["_q"]==i])
    if instances == 0:
        pass
    elif instances >= 2:
        dupeframe = gaia_df[gaia_df["_q"]==i][["_q","_r"]]
        gaia_df.drop(dupeframe[dupeframe["_r"]>min(dupeframe["_r"])].index, inplace=True)

gaia_df.dropna(axis=0, subset=["_r"], inplace=True)
F0 = 2.9979246e-5 * 3631 / (6217.59**2) # Jy -> erg/cm^2/s/Ang
gaia_df["Gflux"] = F0 * 10**(gaia_df["Gmag"]/-2.5)

assert(len(gaia_df) == len(np.unique(gaia_df["_q"]))),"Something went wrong..."

### HMXB crossmatch

- 2CXO J210335.6+454505 with 92%/100% is a HMXB in our TD
- 2CXO J213930.6+565910 with 83%/99% is a HMXB from Simbad, but not in our TD
- 2CXO J112057.1-615500 with 96%/100% is a HMXB from Liu+06 in our TD
- 2CXO J223920.8+611626 with 81%/100% is a HMXB from Liu+06 in our TD
- 2CXO J174445.7-271344 #  not significant with 45%/70% is a HMXB in our TD, but a Be Star in Simbad
- 2CXO J024031.6+611345 #  not significant with 58%/95% is a HMXB from Liu+06
- 2CXO J113106.9-625648 #  not significant with 36%/35% is not in our TD, is a HMXB from Simbad
- 2CXO J130247.6-635008 #  not significant with 53%/56% is not in our TD, is a HMXB from Simbad, 
- 2CXO J134632.5-625523 #  not significant with 44%/47% is not in our TD, is a Be Star from Simbad

In [8]:
HMXBs_BeSS_1as = v.query_region(BeSS, catalog="J/A+A/455/1165/table1", radius="1s")

### XMM, Chandra, and Swift/XRT catalogs

| Band      | Energy bin range |
|-----------|------------------|
| XMM Flux1 | 0.2 - 0.5 keV    |
| XMM Flux2 | 0.5 - 1.0 keV    |
| XMM Flux3 | 1.0 - 2.0 keV    |
| XMM Flux4 | 2.0- 4.5 keV     |
| XMM Flux5 | 4.5 - 12.0 keV   |
| XMM Flux8 | 0.2 - 12.0 keV   |
| XMM Flux9 | 0.5 - 4.5 keV    |
| CXO Fluxu | 0.2 - 0.5 keV    |
| CXO Fluxs | 0.5 - 4.2 keV    |
| CXO Fluxm | 1.2 - 2.0 keV    |
| CXO Fluxh | 2.0 - 7.0 keV    |
| CXO Fluxb | 0.5 - 7.0 keV    |

Load the full catalogs from locally stored copies. Crossmatch from vizier remotely.

In [10]:
cat_ids = ["IX/59/xmm4dr9s","IX/57/csc2master","IX/58/2sxpscle"]
Xmatch = [None, None, None]
for i in range(len(cat_ids)):
    Xmatch[i] = v.query_region(BeSS, catalog=cat_ids[i], radius="1s")
    print("Successfully matched BeSS with",cat_ids[i])
[XMM_xmatch, CSC_xmatch, XRT_xmatch] = Xmatch

Successfully matched BeSS with IX/59/xmm4dr9s
Successfully matched BeSS with IX/57/csc2master
Successfully matched BeSS with IX/58/2sxpscle


In [28]:
Chandra_sources = CSC_xmatch[0]["_q","_2CXO","Fluxb","Fluxh","Fluxm","Fluxs","Fluxu","HRhm","HRhs","HRms",
                                "VaKSb","VaKPb","VaGLb","Vib"]
chandra_bands = {"b":(0.5,7), "h":(2,7), "m":(1.2,2), "s":(0.5,1.2), "u":(0.2,0.5)}

XMM_sources = XMM_xmatch[0]["_q","_4XMM","Flux1","Flux2","Flux3","Flux4","Flux5","Flux8","Flux9",
                            "HR1","HR2","HR3","HR4","Cst","Fvar","e_Fvar","V"]
xmm_bands = dict(zip(list("1234589"),[(0.2,0.5),(0.5,1.0),(1.0,2.0),(2.0,4.5),(4.5,12.0),(0.2,12.0),(0.5,4.5)]))

XRT_sources = XRT_xmatch[0]["_q","_2SXPS","HR1","HR2","FPO0","FPU0","NH1H","Gamma"]

### Compile a master catalog

In [29]:
objs = pd.merge(gaia_df, HMXBs_BeSS_1as[0].to_pandas(), on="_q", how="outer")
xray = pd.merge(XMM_sources.to_pandas(), Chandra_sources.to_pandas(), on="_q", how="outer")
xray = pd.merge(xray, XRT_sources.to_pandas(), on="_q", how="outer")

master = pd.merge(BeSS.to_pandas(), pd.merge(objs, xray, on="_q", how="outer"), left_on="#", right_on="_q", how="outer")

In [31]:
for i in master.index:
    flags = []
    if pd.notna(master.loc[i,"Be star"]):
        flags.append("BeSS")
    if pd.notna(master.loc[i,"EDR3Name"]):
        flags.append("Gaia")
    if pd.notna(master.loc[i,"Name"]):
        flags.append("HMXB")
    if pd.notna(master.loc[i,"_4XMM"]):
        flags.append("XMM")
    if pd.notna(master.loc[i,"_2CXO"]):
        flags.append("Chandra")
    master.loc[i,"flags"] = ",".join(flags)

In [32]:
master.drop(['RA','DEC','_r_x','_RAJ2000_y','_DEJ2000_y','_r_y','RAJ2000','DEJ2000','GLON_y','GLAT_y',"u_Name2"], axis=1, inplace=True)
master.rename(columns={"V_x":"V","Type_x":"SpecType",'_RAJ2000_x':"_RAJ2000",'_DEJ2000_x':"_DEJ2000","GLON_x":"GLON","GLAT_x":"GLAT","Type_y":"Type","V_y":"Vflag"}, inplace=True)

In [42]:
master

,#,Be star,Category,V,SpecType,vsini,Nb of spectra in BeSS,_RAJ2000,_DEJ2000,_q,...,VaGLb,Vib,_2SXPS,HR1_y,HR2_y,FPO0,FPU0,NH1H,Gamma,flags
0,1,BD+62 2346,Classical,9.730,B0Ve,<NA>,3,0.352917,63.504369,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"BeSS,Gaia"
1,2,HD 224905,Classical,8.470,B1Vne,<NA>,85,0.410958,60.449922,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"BeSS,Gaia"
2,3,HD 225095,Classical,7.950,B2IVne,<NA>,234,0.863125,55.550897,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"BeSS,Gaia"
3,4,2 Cet,Classical,4.543,B9IVne,<NA>,95,0.934958,-17.335992,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"BeSS,Gaia"
4,5,10 Cas,Classical,5.567,B9IIIe,125,292,1.610583,64.196169,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"BeSS,Gaia"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2261,2260,BD+65 1970,Classical,10.170,B5e,<NA>,1,359.392083,66.431622,2260.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"BeSS,Gaia"
2262,2261,LQ And,Classical,6.542,B4Vne,300,507,359.693500,46.413178,2261.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"BeSS,Gaia"
2263,2262,HD 224544,Classical,6.524,B6IVe,260,402,359.705167,32.381706,2262.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"BeSS,Gaia"
2264,2263,HD 224599,Classical,9.700,B0.5Vnnpe,<NA>,2,359.812375,60.022481,2263.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"BeSS,Gaia"


In [41]:
master.to_csv("./catalogs/BeSS+Gaia+HMXBs+CSC+XMM+XRT.csv", index=False)